In [1]:
import pandas as pd

# 定义数据文件路径（请根据你的实际情况修改路径）
base_path = "C:/Users/31579/Downloads/archive/ml-100k/"  # 假设数据集文件解压在当前目录下的ml-100k文件夹中

# 1. 转换评分数据 (u.data)
# u.data文件包含用户ID、电影ID、评分和时间戳:cite[10]，列之间通常用制表符分隔:cite[7]
ratings_columns = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(base_path + 'u.data', sep='\t', names=ratings_columns, engine='python')
ratings.to_csv(base_path + 'ratings.csv', index=False)
print("评分文件 (ratings.csv) 已生成。")

# 2. 转换电影数据 (u.item)
# u.item文件包含电影ID、标题、上映日期和流派等信息:cite[10]，列之间用'|'分隔:cite[7]
# 电影数据包含很多列，这里我们只读取前几列并命名，同时为流派列创建列名
movie_columns = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
# 根据数据集说明，后面还有19个流派列:cite[10]
genres = ["unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", 
          "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", 
          "Thriller", "War", "Western"]
all_movie_columns = movie_columns + genres
movies = pd.read_csv(base_path + 'u.item', sep='|', names=all_movie_columns, encoding='latin-1', engine='python')
movies.to_csv(base_path + 'movies.csv', index=False)
print("电影文件 (movies.csv) 已生成。")

# 3. 转换用户数据 (u.user)
# u.user文件包含用户ID、年龄、性别、职业和邮编:cite[10]，列之间用'|'分隔
users_columns = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
users = pd.read_csv(base_path + 'u.user', sep='|', names=users_columns, engine='python')
users.to_csv(base_path + 'users.csv', index=False)
print("用户文件 (users.csv) 已生成。")

print("\n所有文件转换完成！")

评分文件 (ratings.csv) 已生成。
电影文件 (movies.csv) 已生成。
用户文件 (users.csv) 已生成。

所有文件转换完成！


In [2]:
# 导入必要的库
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 加载数据
movies = pd.read_csv('C:/Users/31579/Downloads/archive/ml-100k/movies.csv') # 替换为你的实际文件路径
ratings = pd.read_csv('C:/Users/31579/Downloads/archive/ml-100k/ratings.csv') # 替换为你的实际文件路径

# 查看数据长什么样
print("电影数据：")
print(movies.head()) # 显示前5行
print("\n评分数据：")
print(ratings.head())

电影数据：
   movie_id              title release_date  video_release_date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995                 NaN   

                                            imdb_url  unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0       0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0       1   
4  http://us.imdb.com/M/title-exact?Copycat%20(1995)        0       0   

   Adventure  Animation  Children's  ...  Fantasy  Film-Noir  Horror  Musical  \
0          0          1           1  ...        0          0       0   

In [3]:
# 检查缺失值
print(ratings.isnull().sum())

# 创建一个“用户-电影”评分矩阵
# 这个矩阵的行是用户，列是电影，值是评分
ratings_matrix = ratings.pivot_table(index='user_id', columns='movie_id', values='rating')

# 查看矩阵形状
print(f"评分矩阵的形状：{ratings_matrix.shape}")

# 用0填充缺失值（因为协同过滤计算需要数值）
ratings_matrix.fillna(0, inplace=True)

# 再次查看填充后的矩阵
print(ratings_matrix.head())

user_id      0
movie_id     0
rating       0
timestamp    0
dtype: int64
评分矩阵的形状：(943, 1682)
movie_id  1     2     3     4     5     6     7     8     9     10    ...  \
user_id                                                               ...   
1          5.0   3.0   4.0   3.0   3.0   5.0   4.0   1.0   5.0   3.0  ...   
2          4.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   2.0  ...   
3          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
4          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
5          4.0   3.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   

movie_id  1673  1674  1675  1676  1677  1678  1679  1680  1681  1682  
user_id                                                               
1          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4          0

In [4]:
# 计算电影之间的余弦相似度
# 这步可能会花点时间，因为它要计算所有电影两两之间的相似度
movie_similarity = cosine_similarity(ratings_matrix.T) # .T 表示对矩阵进行转置，让电影变成行

# 将相似度矩阵转换为DataFrame，方便查看
movie_similarity_df = pd.DataFrame(movie_similarity, index=ratings_matrix.columns, columns=ratings_matrix.columns)
print(movie_similarity_df.head())

movie_id      1         2         3         4         5         6     \
movie_id                                                               
1         1.000000  0.402382  0.330245  0.454938  0.286714  0.116344   
2         0.402382  1.000000  0.273069  0.502571  0.318836  0.083563   
3         0.330245  0.273069  1.000000  0.324866  0.212957  0.106722   
4         0.454938  0.502571  0.324866  1.000000  0.334239  0.090308   
5         0.286714  0.318836  0.212957  0.334239  1.000000  0.037299   

movie_id      7         8         9         10    ...      1673  1674  \
movie_id                                          ...                   
1         0.620979  0.481114  0.496288  0.273935  ...  0.035387   0.0   
2         0.383403  0.337002  0.255252  0.171082  ...  0.000000   0.0   
3         0.372921  0.200794  0.273669  0.158104  ...  0.000000   0.0   
4         0.489283  0.490236  0.419044  0.252561  ...  0.000000   0.0   
5         0.334769  0.259161  0.272448  0.055453  ...  0.

In [5]:
def get_similar_movies(movie_name, min_ratings=10):
    # 通过电影名找到它的ID
    movie_id = movies[movies['title'] == movie_name]['movie_id'].values[0]

    # 从相似度矩阵中获取该电影与其他所有电影的相似度分数
    similarity_scores = movie_similarity_df[movie_id]

    # 将这些分数与原始电影数据合并，方便看到电影名
    similar_movies = pd.DataFrame({
        'movie_id': similarity_scores.index,
        'similarity': similarity_scores.values
    }).merge(movies, on='movie_id')

    # 我们可以简单地按相似度分数从高到低排序，并返回前10个
    return similar_movies.sort_values(by='similarity', ascending=False).head(10)

# 输入你喜欢的电影，看看推荐结果！
your_favorite_movie = "Forrest Gump (1994)" # 请确保电影名与数据集中的完全一致
recommendations = get_similar_movies(your_favorite_movie)
print(f"因为你喜欢《{your_favorite_movie}》，所以我们为你推荐：")
print(recommendations[['title']]) # 只打印电影名和类型    

因为你喜欢《Forrest Gump (1994)》，所以我们为你推荐：
                                 title
68                 Forrest Gump (1994)
422  E.T. the Extra-Terrestrial (1982)
27                    Apollo 13 (1995)
173     Raiders of the Lost Ark (1981)
96           Dances with Wolves (1990)
203          Back to the Future (1985)
195          Dead Poets Society (1989)
21                   Braveheart (1995)
97    Silence of the Lambs, The (1991)
201               Groundhog Day (1993)


In [6]:
streamlit run app.py

SyntaxError: invalid syntax (3737097518.py, line 1)